In [84]:
# Import all the necessary libraries
import PIL
import numpy as np
import torch
import sys
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import datasets, transforms
from sklearn.metrics import roc_auc_score

import time

In [259]:
# Hyperparameters
hyper = {
    "task": "Classification",
    "bottleneckSetting": [[1, 16, 1, 1], # t, c, n, s
                          [6, 24, 2, 1],
                          [6, 32, 3, 2],
                          [6, 64, 4, 1],
                          [6, 96, 3, 2],
                          [6, 160, 3, 1],
                          [6, 320, 1, 1]],
    "nEpochs":50,
    "batchSize":256,
    "lr":0.001,#1e-3,
    "dataPath": "./11-785hw2p2-s20",
    "checkpoingPath": "./checkpoint/ContContInitWeight_BaselineSGD_StepLR_Epoch4.txt",
    "classifyTestImgFolderPath": "./11-785hw2p2-s20/test_classification/medium/",
    "classifyTestListPath": "./11-785hw2p2-s20/test_order_classification.txt",
    "verifyImgFolderPath": "./11-785hw2p2-s20/validation_verification/",
    "verifyPairListPath": "./11-785hw2p2-s20/validation_trials_verification.txt",
    "verifyTestPairListPath": "./11-785hw2p2-s20/test_trials_verification_student.txt",
    "verifyTestImgFolderPath": "./11-785hw2p2-s20/test_verification/",
    "weightDirName": "./checkpoint/",
    "testClassLabelName":"./output/test_class_labels.npy",
    "testClassLabelCSVfn":"./output/test_class_labels.csv",
    "testVeriLabelName":"./output/test_veri_labels.npy",
    "testVeriLabelCSVfn":"./output/test_veri_labels.csv",
}

In [251]:
class MyDatasetTestClassify(Dataset):
    # Create custom Dataset
    def __init__(self, testFN, testImgFolderPath):
        self.imgFolderPath = testImgFolderPath
        with open(testFN) as f:
            self.fileList = [line.rstrip() for line in f]
    def __len__(self):
        return len(self.fileList)
    def __getitem__(self, idx):
        img = PIL.Image.open(self.imgFolderPath + self.fileList[idx])
        img = transforms.ToTensor()(img)
        return img, -1
    def getFileList(self):
        return self.fileList

In [252]:
class MyDatasetVerify(Dataset):
    # Create dataset for validation of verification tast
    def __init__(self, pairFN, imgFolderPath):
        self.imgFolderPath = imgFolderPath
        with open(pairFN) as f:
            self.pairList = [line.rstrip() for line in f]
    def __len__(self):
        return len(self.pairList)
    def __getitem__(self, idx):
        items = self.pairList[idx].split()
        fn1, fn2 = items[0], items[1]
        img1 = PIL.Image.open(self.imgFolderPath + fn1)
        img2 = PIL.Image.open(self.imgFolderPath + fn2)
        img1 = transforms.ToTensor()(img1)
        img2 = transforms.ToTensor()(img2)
        if len(items) == 3: # validation
            return img1, img2, int(items[2])
        else: # test
            return img1, img2, -1
    def getPairList(self):
        return self.pairList
        

In [253]:
def conv2d_3x3_bn_relu(inChannel, outChannel, stride):
    return nn.Sequential(
        nn.Conv2d(inChannel, outChannel, 3, stride, 1, bias=False),
        nn.BatchNorm2d(outChannel),
        nn.ReLU6(inplace=True)
    )
def conv2d_1x1_bn_relu(inChannel, outChannel):
    return nn.Sequential(
        nn.Conv2d(inChannel, outChannel, 1, 1, 0, bias=False),
        nn.BatchNorm2d(outChannel),
        nn.ReLU6(inplace=True)
    )

In [254]:
class BottleNeck(nn.Module):
    def __init__(self, inChannel, outChannel, stride, expandT):
        super(BottleNeck, self).__init__()
        self.stride = stride
        self.shouldSkip = self.stride == 1 and inChannel == outChannel
        hiddenDim = int(inChannel * expandT)
        self.conv = nn.Sequential(
            # 1 x 1 expansion layer + bn + ReLU6
            nn.Conv2d(inChannel, hiddenDim, 1, 1, 0, bias=False),
            nn.BatchNorm2d(hiddenDim),
            nn.ReLU6(inplace=True),
            # 3 x 3 depthwise conv + bn + ReLU6
            nn.Conv2d(hiddenDim, hiddenDim, 3, self.stride, 1, groups=hiddenDim, bias=False),
            nn.BatchNorm2d(hiddenDim),
            nn.ReLU6(inplace=True),
            # 1 x 1 projection layer + bn
            nn.Conv2d(hiddenDim, outChannel, 1, 1, 0, bias=False),
            nn.BatchNorm2d(outChannel)
        )
    
    def forward(self, x):
        # only skip connnection when stride==1 and inChannel==outChannel
        if self.shouldSkip:
            return x + self.conv(x)
        else:
            return self.conv(x)

In [255]:
class Network(nn.Module):
    def __init__(self, inputSize, bottlesSetting, numClasses, feat_dim=1280):
        super(Network, self).__init__()
        block = BottleNeck
        firstChannel = 32
        lastChannel = 1280
        blocks = [conv2d_3x3_bn_relu(3, firstChannel, 1)]
        # build MobileNet bottlenecks
        bottleInChannel = firstChannel
        for t, c, n, s in bottlesSetting:
            bottleOutChannel = c
            for i in range(n):
                if i == 0: # the first layer in a sequence
                    blocks.append(block(bottleInChannel, bottleOutChannel, s, t))
                else:
                    blocks.append(block(bottleInChannel, bottleOutChannel, 1, t))
                bottleInChannel = bottleOutChannel
        # build the conv2d 1x1 layer
        blocks.append(conv2d_1x1_bn_relu(bottleInChannel, lastChannel))
        self.net = nn.Sequential(*blocks)
        
        # built classifier
        self.classifier = nn.Linear(lastChannel, numClasses)
#         self.classifier = nn.Sequential(
#             nn.Dropout(0.2),
#             nn.Linear(lastChannel, numClasses)
#         )
         
    def forward(self, x):
        x = self.net(x)
        output = nn.functional.adaptive_avg_pool2d(x, 1).reshape(x.shape[0], -1) # flatten
        classification_out = self.classifier(output)
        embedding_out = output
        return embedding_out, classification_out
        
def init_weights(m):
    if type(m) == nn.Conv2d:
        nn.init.kaiming_normal_(m.weight, mode='fan_out')
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif type(m) == nn.BatchNorm2d:
        nn.init.ones_(m.weight)
        nn.init.zeros_(m.bias)
    elif type(m) == nn.Linear:
        nn.init.normal_(m.weight, 0, 0.01)
        nn.init.zeros_(m.bias)

In [256]:
def parseData(dataDir):
    img_list = []
    for root, directories, filenames in os.walk(dataDir):
        for filename in filenames:
            if filename.endswith(".jpg"):
                filei = os.path.join(root, filename)
                img_list.append(filei)
    print("{} # Images".format(len(img_list)))
    return img_list

In [257]:
def getLoaders(trainDS, devDS, testDS, batchS):
    print("*** Create data loader ***")
    
    # Train
    loader_args = dict(shuffle=True, batch_size=batchS, num_workers=8, pin_memory=True)
    train_loader = DataLoader(trainDS, **loader_args)
    
    # Dev
    dev_loader = DataLoader(devDS, **loader_args)
    
    # Test
    test_loader_args = dict(shuffle=False, batch_size=100, num_workers=1, pin_memory=True)
    test_loader = DataLoader(testDS, **test_loader_args)
    
    return train_loader, dev_loader, test_loader

In [258]:
def train_epoch(model, data_loader, criterion, optimizer, epoch): #task="Classification")
    model.train()
    running_loss = 0.0
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        data, target = data.cuda(), target.cuda()

        outputs = model(data)[1]
        loss = criterion(outputs, target)
        running_loss += loss.item()
        
        loss.backward()
        optimizer.step()

        if batch_idx % 1000 == 0:
            print("Epoch: {}\tBatch: {}\tTimestamp: {}".format(epoch, batch_idx, time.time() - start_time))
        
        # clear computation cache
        torch.cuda.empty_cache()
        del data
        del target
        del loss
    end_time = time.time()
    running_loss = running_loss / len(data_loader)
    return running_loss

def testClassify(model, test_loader, epoch):
    with torch.no_grad():
        model.eval()
        start_time = time.time()
        running_loss = 0.0
        correct_predictions = 0.0
        total_predictions = 0.0
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.cuda(), target.cuda()

            outputs = model(data.float())[1]
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, target.long()).detach()
            total_predictions += target.size(0)
            correct_predictions += (predicted==target).sum().item()
            running_loss += loss.item()
            if batch_idx % 500 == 0:
                print("Epoch: {}\tBatch: {}\tTimestamp: {}".format(epoch, batch_idx, time.time()-start_time))
            del data
            del target
        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        return running_loss, acc


def testVerify(model, vLoader):
    similarity = np.array([])
    true = np.array([])
    start_time = time.time()
    with torch.no_grad():
        start_time = time.time()
        for batch_idx, (imgs1, imgs2, targets) in enumerate(vLoader):
            imgs1, imgs2, targets = imgs1.cuda(), imgs2.cuda(), targets.cuda()
            # find cos similarity between embeddings
            imgs1Embed = model(imgs1.float())[0]
            imgs2Embed = model(imgs2.float())[0]
            sim = F.cosine_similarity(imgs1Embed, imgs2Embed) 
            similarity = np.concatenate((similarity, sim.cpu().numpy().reshape(-1)))
            true = np.concatenate((true, targets.cpu().numpy().reshape(-1)))
            if batch_idx % 100 == 0:
                print("Batch: {}\t Timestamp:{}".format(batch_idx, time.time()-start_time))
            del imgs1
            del imgs2
            del targets
    return similarity, true
            

def predictLabels(model, test_loader):
    with torch.no_grad():
        model.eval()
        res = np.array([])
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.cuda(), target.cuda()

            outputs = model(data)[1]
            _, predicted = torch.max(outputs.data, dim=1)
            res = np.concatenate((res, predicted.cpu().numpy().reshape(-1)))
            del data
            del target
    return res
        

In [239]:
dataFolder = hyper['dataPath']
wegithDirName = hyper['weightDirName']
cuda = torch.cuda.is_available()

print("*** Load raw data ***")
train = datasets.ImageFolder(root=dataFolder+"/train_data/medium",
                            transform=transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor()]))

dev = datasets.ImageFolder(root=dataFolder+"/validation_classification/medium",
                           transform=transforms.ToTensor())

# Load custom dataset for test since it does not follow ImageFolder structure
test = MyDatasetTestClassify(hyper["classifyTestListPath"], hyper["classifyTestImgFolderPath"])

print("train data stat: {} images \t {} classes".format(train.__len__(), len(train.classes)))
print("dev data stat: {} images \t {} classes".format(dev.__len__(), len(dev.classes)))
print("test data stat: {} images".format(test.__len__()))

# Get data loaders
train_loader, dev_loader, test_loader = getLoaders(train, dev, test, hyper["batchSize"])

*** Load raw data ***
train data stat: 822155 images 	 2300 classes
dev data stat: 4600 images 	 2300 classes
test data stat: 4600 images
*** Create data loader ***


In [139]:
train.__getitem__(1)[0].shape

torch.Size([3, 32, 32])

In [260]:
# Create the model and define the Loss and Optimizer
print("*** Create the model and define  Loss and Optimizer ***")
inputSize = train.__len__()         # number of train input images
outputSize = len(train.classes)     # number of unique face classes
model = Network(inputSize, hyper["bottleneckSetting"], outputSize)
# model.apply(init_weights)
checkpoint = torch.load(hyper["checkpoingPath"])
model.load_state_dict(checkpoint["model_state_dict"])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=hyper["lr"], momentum=0.9,\
                      nesterov=True, weight_decay=5e-4)
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.8, patience=1)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.9, step_size=1)
device = torch.device("cuda" if cuda else "cpu")
model.cuda()
print(model)

*** Create the model and define  Loss and Optimizer ***
Network(
  (net): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): BottleNeck(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU6(inplace=True)
        (6): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (7): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): BottleNeck

In [262]:
# Train the model for N epochs
print("*** Train the model for N epochs ***")
Train_loss = []
Train_acc = []
Test_loss = []
Test_acc = []
for i in range(hyper["nEpochs"]):
    for prarm_group in optimizer.param_groups:
        print("Current lr: \t{}".format(prarm_group["lr"]))
    startTime = time.time()
    print("Train\tEpoch: {}".format(i))
    train_loss = train_epoch(model, train_loader, criterion, optimizer, i)
    if hyper["task"] == "Classification":
        print("Classify Train \tEpoch: {}".format(i))
        train_loss, train_acc = testClassify(model, train_loader, i)
        print("Classify Dev \tEpoch: {}".format(i))
        dev_loss, dev_acc = testClassify(model, dev_loader, i)
        print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
              format(train_loss, train_acc, dev_loss, dev_acc))
    else:
        print("Verification task")
    scheduler.step(dev_loss)
    Train_loss.append(train_loss)
    Train_acc.append(train_acc)
    Test_loss.append(dev_loss)
    Test_acc.append(dev_acc)
    print("*** Saving Checkpoint ***")
    path = "{}ContContContInitWeight_BaselineSGD_StepLR_Epoch{}.txt".format(wegithDirName, i)
    torch.save({
        "epoch": i,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'train_loss': train_loss,
        "train_acc": train_acc,
        'dev_loss':dev_loss,
        'dev_acc': dev_acc
    }, path)
    print("="*20 + " Epoch {} took {}s".format(i, time.time()-startTime) + "="*20)

*** Train the model for N epochs ***
Current lr: 	0.001
Train	Epoch: 0
Epoch: 0	Batch: 0	Timestamp: 0.9484448432922363
Epoch: 0	Batch: 1000	Timestamp: 497.70069432258606
Epoch: 0	Batch: 2000	Timestamp: 1011.2534255981445
Epoch: 0	Batch: 3000	Timestamp: 1526.0770795345306
Classify Train 	Epoch: 0
Epoch: 0	Batch: 0	Timestamp: 0.6695742607116699
Epoch: 0	Batch: 500	Timestamp: 53.67986011505127
Epoch: 0	Batch: 1000	Timestamp: 106.7500433921814
Epoch: 0	Batch: 1500	Timestamp: 159.83489513397217
Epoch: 0	Batch: 2000	Timestamp: 212.99435114860535
Epoch: 0	Batch: 2500	Timestamp: 266.1447112560272
Epoch: 0	Batch: 3000	Timestamp: 319.3462505340576
Classify Dev 	Epoch: 0
Epoch: 0	Batch: 0	Timestamp: 0.5989861488342285
Train Loss: 0.7964	Train Accuracy: 84.6764	Val Loss: 1.3993	Val Accuracy: 71.0217
*** Saving Checkpoint ***
==================== Epoch 0 took 1979.2950837612152s====================
Current lr: 	0.001
Train	Epoch: 1
Epoch: 1	Batch: 0	Timestamp: 0.8218810558319092
Epoch: 1	Batch: 100

KeyboardInterrupt: 

In [152]:
Test_acc

[72.15217391304348,
 71.78260869565217,
 71.67391304347827,
 72.34782608695653,
 72.60869565217392,
 72.34782608695653,
 72.54347826086956,
 72.26086956521739,
 72.3913043478261]

In [181]:
# Writeout test labels
labels = predictLabels(model, test_loader)
labels = list(map(int, labels))
idxs = np.array(test.getFileList())
labels = np.array(labels)
# create mappings of file set labels to true labels
alphabetSorted = sorted([str(x) for x in range(0, 2300)])
filesetTrueLabelTuple = [(i, int(alphabetSorted[i])) for i in range(len(alphabetSorted))]
mapping = dict(filesetTrueLabelTuple)
labels = np.array(list(map(mapping.get, labels)))
np.save(hyper["testClassLabelName"], labels)
df = pd.DataFrame({"Id" : idxs, "Category" : labels})
df.to_csv(hyper["testClassLabelCSVfn"], index=False)

In [218]:
# Read in verification pairs for validation
verifyData_valid = MyDatasetVerify(hyper["verifyPairListPath"], hyper["verifyImgFolderPath"])
verify_loader_args_valid = dict(shuffle=False, batch_size=200, num_workers=8, pin_memory=True)
verify_loader_valid = DataLoader(verifyData_valid, **verify_loader_args_valid)

In [219]:
print("Verification validation data stat: {} pairs \t".format(verifyData_valid.__len__()))

Verification validation data stat: 100000 pairs 	


In [224]:
# Calculate simliarity score
cosScore_valid, trueScore_valid = testVerify(model, verify_loader_valid)

# Report AUC
auc = roc_auc_score(trueScore_valid, cosScore_valid)
print("*** AUC: {} ***".format(auc))

Batch: 0	 Timestamp:0.8667874336242676
Batch: 100	 Timestamp:16.919691801071167
Batch: 200	 Timestamp:33.428550004959106
Batch: 300	 Timestamp:50.103861570358276
Batch: 400	 Timestamp:66.74586343765259
*** AUC: 0.9401668958798193 ***


In [243]:
# Read in verification pairs for test
verifyData_test = MyDatasetVerify(hyper["verifyTestPairListPath"], hyper["verifyTestImgFolderPath"])
verify_loader_args_test = dict(shuffle=False, batch_size=300, num_workers=8, pin_memory=True)
verify_loader_test = DataLoader(verifyData_test, **verify_loader_args_test)

In [246]:
print("Verification test data stat: {} pairs \t".format(verifyData_test.__len__()))

Verification test data stat: 899965 pairs 	


In [245]:
# Calculate similarity score
cosScore_test, _ = testVerify(model, verify_loader_test)


Batch: 0	 Timestamp:1.4902324676513672
Batch: 100	 Timestamp:28.6463360786438
Batch: 200	 Timestamp:56.59016489982605
Batch: 300	 Timestamp:84.8301374912262
Batch: 400	 Timestamp:112.83456826210022
Batch: 500	 Timestamp:140.91132235527039
Batch: 600	 Timestamp:169.0271396636963
Batch: 700	 Timestamp:197.16711831092834
Batch: 800	 Timestamp:225.24022388458252
Batch: 900	 Timestamp:253.30145454406738
Batch: 1000	 Timestamp:281.33954644203186
Batch: 1100	 Timestamp:309.18132305145264
Batch: 1200	 Timestamp:337.0060164928436
Batch: 1300	 Timestamp:364.8084807395935
Batch: 1400	 Timestamp:392.5616104602814
Batch: 1500	 Timestamp:420.29006719589233
Batch: 1600	 Timestamp:448.0446226596832
Batch: 1700	 Timestamp:475.78411889076233
Batch: 1800	 Timestamp:503.5323119163513
Batch: 1900	 Timestamp:531.2998230457306
Batch: 2000	 Timestamp:559.0744941234589
Batch: 2100	 Timestamp:586.861249923706
Batch: 2200	 Timestamp:614.6070256233215
Batch: 2300	 Timestamp:642.3246262073517
Batch: 2400	 Timestam

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f80d3694cf8>>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f8136210198>>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/pytho

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f80d3694cf8>>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/s

Batch: 2700	 Timestamp:753.4510560035706
Batch: 2800	 Timestamp:781.204389333725
Batch: 2900	 Timestamp:808.9699437618256


In [247]:
len(cosScore_test)

899965

In [248]:
# Save predictied similarity
cosScore_test = np.array(cosScore_test)
np.save(hyper["testVeriLabelName"], cosScore_test)
trial = np.array(verifyData_test.getPairList())
df = pd.DataFrame({"trial" : trial, "score" : cosScore_test})
df.to_csv(hyper["testVeriLabelCSVfn"], index=False)

In [249]:
df.head()

,trial,score
0,262615.jpg 207587.jpg,0.357082
1,120800.jpg 162540.jpg,0.367260
2,200386.jpg 117646.jpg,0.414564
3,268346.jpg 264478.jpg,0.296321
4,171295.jpg 143107.jpg,0.337177


In [140]:
# Load data
def main(hyper):
    dataFolder = hyper['dataPath']
    wegithDirName = hyper['weightDirName']
    cuda = torch.cuda.is_available()

    print("*** Load raw data ***")
    train = datasets.ImageFolder(root=dataFolder+"/train_data/medium",
                                transform=transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor()]))

    dev = datasets.ImageFolder(root=dataFolder+"/validation_classification/medium",
                               transform=transforms.ToTensor())

    # Load custom dataset for test since it does not follow ImageFolder structure
    test = MyDatasetTestClassify(hyper["classifyTestListPath"], hyper["classifyTestImgFolderPath"])

    print("train data stat: {} images \t {} classes".format(train.__len__(), len(train.classes)))
    print("dev data stat: {} images \t {} classes".format(dev.__len__(), len(dev.classes)))
    print("test data stat: {} images".format(train.__len__()))

    # Get data loaders
    train_loader, dev_loader, test_loader = getLoaders(train, dev, test, hyper["batchSize"])
    
    # Create the model and define the Loss and Optimizer
    print("*** Create the model and define  Loss and Optimizer ***")
    inputSize = train.__len__()         # number of train input images
    outputSize = len(train.classes)     # number of unique face classes
    model = Network(inputSize, hyper["hiddenDims"], outputSize)
    checkpoint = torch.load(hyper["checkpoingPath"])
    # model.load_state_dict(checkpoint["model_state_dict"])
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=hyper["lr"], momentum=0.9,\
                          nesterov=True, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", patience=2)
    # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.9, step_size=1)
    device = torch.device("cuda" if cuda else "cpu")
    model.cuda()
    print(model)
    
    # Train the model for N epochs
    print("*** Train the model for N epochs ***")
    Train_loss = []
    Test_loss = []
    Test_acc = []
    for i in range(hyper["nEpochs"]):
        print("Training\tEpoch: {}".format(i))
        train_loss = train_epoch(model, train_loader, criterion, optimizer)
        print("Dev\tEpoch: {}".format(i))
        if hyper["task"] == "Classification":
            train_loss, train_acc = testClassify(model, train_loader)
            dev_loss, dev_acc = testClassify(model, dev_loader)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc, val_loss, val_acc))
        else:
            print("Verification task")
        scheduler.step(dev_loss)
        Train_loss.append(train_loss)
        Test_loss.append(dev_loss)
        Test_acc.append(dev_acc)
        print("="*20)
        print("*** Saving Checkpoing ***")
        path = "{}InitWeight_BaselineSGD_StepLR_Epoch{}.txt".format(wegithDirName, i)
        torch.save({
            "epoch": i,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'dev_loss':dev_loss,
            'dev_acc': dev_acc
        }, path)
        
    # Visualizing Training and Validation data
    print("*** Visualizing Training and Validation Data ***")
    plt.title('Training Loss')
    plt.xlabel('Epoch Number')
    plt.ylabel('Loss')
    plt.plot(Train_loss)
    plt.savefig("Train_Vis.png")
    
    plt.title('Dev Accuracy')
    plt.xlabel('Epoch Number')
    plt.ylabel('Accuracy (%)')
    plt.plot(Test_acc)
    plt.savefig("Dev_Vis.png")
    
    # Writeout test labels
    labels = predictLabels(model, test_loader)
    labels = list(map(int, labels))
    idxs = np.array(test.getFileList())
    labels = np.array(labels)
    # create mappings of file set labels to true labels
    alphabetSorted = sorted([str(x) for x in range(0, 2300)])
    filesetTrueLabelTuple = [(i, int(alphabetSorted[i])) for i in range(len(alphabetSorted))]
    mapping = dict(filesetTrueLabelTuple)
    labels = np.array(list(map(mapping.get, labels)))
    np.save(hyper["testLabelName"], labels)
    df = pd.DataFrame({"Id" : idxs, "Category" : labels})
    df.to_csv(hyper["testLabelCSVfn"], index=False)

In [ ]:
imgs1E = torch.abs(torch.randn(2, 1, 50))
imgs2E = torch.abs(torch.randn(2, 1, 50))
res1 = F.cosine_similarity(imgs1E, imgs2E, 1)
print(res1)